In [10]:
import numpy
import pandas as pd
import random
import math
import scipy
from scipy.stats import ks_2samp
from scipy.stats import norm,gamma

In [1]:
pip install minepy

  Using cached minepy-1.2.6.tar.gz (496 kB)
  Created wheel for minepy: filename=minepy-1.2.6-cp39-cp39-win_amd64.whl size=56410 sha256=043eb1efb22ba57efe9f440f6be664d1f5acc0da1f75c9b22dd42e024c839238
  Stored in directory: c:\users\krishna\appdata\local\pip\cache\wheels\36\71\63\ac6efc0d0f43ad93398a7b25b6ef3b3aacf109e7dc5afe588f
Successfully built minepy
Note: you may need to restart the kernel to use updated packages.


In [4]:
comp_file_black = pd.read_csv('E:\\Comparator model - Sheet1 (5).csv')
comp_file_black = comp_file_black.where(pd.notnull(comp_file_black),None)
comp_file_black.head(10)

,index,column 1,column 2,column 3,column 4,column 5
0,distribution,norm,norm,None,None,None
1,parameters,"0,6","0,5",None,None,None
2,relationship,None,None,[1],[2],"[3,4]"
3,bias,"norm.rvs(0,0.1,size=no_of_data)","norm.rvs(0,0.1,size=no_of_data)","norm.rvs(0,0.01,size=no_of_data)","norm.rvs(0,0.01,size=no_of_data)",None
4,generator,"norm.rvs(0,6,size=no_of_data)","norm.rvs(0,5,size=no_of_data)",[math.sin(i) for i in list[0]],[math.cos(i) for i in list[0]],"[i/j for i,j in zip(list[0],list[1])]"


In [5]:
comp_file_white = pd.read_csv('E:\\Comparator model - Sheet1 (4).csv')
comp_file_white = comp_file_white.where(pd.notnull(comp_file_white),None)
comp_file_white.head(10)

,index,column 1,column 2,column 3,column 4,column 5
0,distribution,norm,None,gamma,None,None
1,parameters,"0,5",None,0.6,None,None
2,relationship,None,[1],None,[3],"[1,3]"
3,bias,None,"scipy.stats.norm.rvs(0,0.1,size=no_of_data)",None,"scipy.stats.norm.rvs(0,0.001,size=no_of_data)",None
4,generator,"scipy.stats.norm.rvs(0,5,size=no_of_data)",[2*i for i in list [0]],"scipy.stats.gamma.rvs(0.6,size=no_of_data)",[i**2 for i in list [0]],"[i+j for i,j in zip(list [0],list [1])]"


In [6]:
def data_generator(comp_file,no_of_data):
    blackbox_dict={}
    def add_list(list1,list2):
        list3 = [i+j for i,j in zip(list1,list2)]
        return list3

    list_columns = [[]]*(len(comp_file.columns)-1)
    for i in range(5):
        if comp_file.iloc[2,i+1] is not None:
            relation = eval(comp_file.iloc[2,i+1])
            list = [[]]*len(relation)
            for j in range(len(relation)):
                list[j] = list_columns[relation[j]-1]
        list_columns[i] = eval(comp_file.iloc[4,i+1])
        if comp_file.iloc[3,i+1] is not None:
            bias_list = eval(comp_file.iloc[3,i+1])
            list_columns[i] = add_list(list_columns[i],bias_list)
        key = 'column {}'.format(i+1)
        blackbox_dict[key] = list_columns[i]

    df = pd.DataFrame(blackbox_dict)
    return df

In [69]:
def find_corr(matrix,corr_percent):
    label=0
    for i in range(len(matrix)):
        for j in range(i+1,len(matrix.columns)):
            if matrix.iloc[i,j]>=corr_percent:
                print('correlation exists between column ',i+1,'and column ',j+1)
                label=1
    if label==0:
        print('No correlations exist')

In [8]:
df_black = data_generator(comp_file_black,1000)
df_black.head(10)

,column 1,column 2,column 3,column 4,column 5
0,-3.837752,-2.471255,0.649176,-0.795126,-0.816445
1,4.048108,-2.110358,-0.796897,-0.510015,1.562497
2,-2.522109,1.055837,-0.587633,0.485776,-1.209678
3,7.434469,-3.575608,0.919466,-0.907936,-1.012699
4,-1.723982,-1.061364,-1.009906,0.491682,-2.053982
5,1.302806,-8.453861,0.962640,-0.552883,-1.741128
6,2.278273,2.959305,0.759570,-0.966663,-0.785765
7,-7.113577,5.895277,-0.725953,0.928049,-0.782235
8,2.590206,0.750749,0.515506,0.724373,0.711659
9,-2.018857,0.210712,-0.881915,0.978663,-0.901143


In [11]:
df_white = data_generator(comp_file_white,1000)
df_white.head(10)

,column 1,column 2,column 3,column 4,column 5
0,-7.223710,-14.445866,3.816796,14.568348,-3.406914
1,2.421467,4.895758,0.550185,0.302931,2.971652
2,6.547487,13.008235,0.026098,0.001000,6.573586
3,6.304540,12.461752,1.126604,1.269722,7.431144
4,1.621062,3.324096,0.285003,0.080784,1.906066
5,1.057070,2.072930,0.002506,-0.000211,1.059576
6,-0.615543,-1.071029,0.459999,0.209178,-0.155544
7,-0.463279,-0.850472,0.397855,0.157889,-0.065424
8,0.436321,0.837945,0.319850,0.104348,0.756171
9,1.746734,3.525238,0.175700,0.030943,1.922434


In [12]:
# pearson r correlation
df_black.corr()

,column 1,column 2,column 3,column 4,column 5
column 1,1.000000,-0.024476,-0.009936,0.020688,-0.030695
column 2,-0.024476,1.000000,-0.033220,0.108627,-0.013466
column 3,-0.009936,-0.033220,1.000000,0.015028,0.019240
column 4,0.020688,0.108627,0.015028,1.000000,0.002297
column 5,-0.030695,-0.013466,0.019240,0.002297,1.000000


In [13]:
df_white.corr()

,column 1,column 2,column 3,column 4,column 5
column 1,1.000000,0.999940,-0.054151,-0.033283,0.986455
column 2,0.999940,1.000000,-0.054377,-0.033717,0.986358
column 3,-0.054151,-0.054377,1.000000,0.871725,0.110376
column 4,-0.033283,-0.033717,0.871725,1.000000,0.110074
column 5,0.986455,0.986358,0.110376,0.110074,1.000000


In [14]:
find_corr(df_black.corr(),0.5)

No linear correlations exist


In [15]:
find_corr(df_white.corr(),0.5)

linear correlation exists between column  1 and column  2
linear correlation exists between column  1 and column  5
linear correlation exists between column  2 and column  5
linear correlation exists between column  3 and column  4


In [68]:
# mic
from minepy import MINE
from sklearn import preprocessing
mine = MINE(alpha=0.6,c=15)
scores=[[]]*(df_black.shape[1])
for i in range(df_black.shape[1]):
    dummy_list=[]
    for j in range(df_black.shape[1]):
        X = df_black.iloc[:,i]
        Y = df_black.iloc[:,j]
        X_scaled = preprocessing.scale(X)
        Y_scaled = preprocessing.scale(Y)
        mine.compute_score(X,Y)
        corr = float(mine.mic())
        dummy_list.append(corr)
    scores[i]=dummy_list
index_row=[]
df_corr_dict={}
for g in range(df_black.shape[1]):
    key = 'column {}'.format(g+1)
    df_corr_dict[key] = scores[g]
    index_row.append(key)

df_corr = pd.DataFrame(df_corr_dict,index = index_row)
df_corr.head(10)

,column 1,column 2,column 3,column 4,column 5
column 1,1.000000,0.130718,1.000000,0.132869,0.340046
column 2,0.130718,1.000000,0.121266,1.000000,0.218795
column 3,1.000000,0.121266,1.000000,0.133193,0.386426
column 4,0.132869,1.000000,0.133193,1.000000,0.334662
column 5,0.340046,0.218795,0.386426,0.334662,1.000000


In [70]:
find_corr(df_corr,0.9)

correlation exists between column  1 and column  3
correlation exists between column  2 and column  4


In [196]:
pv=[]
for i in range(5):
    kstat,pvalue = ks_2samp(df_black.iloc[:,i],df_white.iloc[:,i])
    pv.append(pvalue)
pv

[0.002393409648884169,
 1.5702401851850447e-15,
 9.185171040635447e-122,
 2.1489890211363644e-131,
 5.77199391392635e-29]

In [72]:
def mic_corr(df):
    from minepy import MINE
    from sklearn import preprocessing
    mine = MINE(alpha=0.6,c=15)
    scores=[[]]*(df.shape[1])
    for i in range(df.shape[1]):
        dummy_list=[]
        for j in range(df.shape[1]):
            X = df.iloc[:,i]
            Y = df.iloc[:,j]
            X_scaled = preprocessing.scale(X)
            Y_scaled = preprocessing.scale(Y)
            mine.compute_score(X,Y)
            corr = float(mine.mic())
            dummy_list.append(corr)
        scores[i]=dummy_list
    
    df_corr_dict={}
    for g in range(df.shape[1]):
        key = df.columns[g]
        df_corr_dict[key] = scores[g]
        

    df_corr = pd.DataFrame(df_corr_dict,index = df.columns)
    return df_corr

In [73]:
mic_corr(df_white)

,column 1,column 2,column 3,column 4,column 5
column 1,1.000000,1.000000,0.123935,0.124834,0.983536
column 2,1.000000,1.000000,0.132372,0.132165,0.975805
column 3,0.123935,0.132372,1.000000,1.000000,0.134502
column 4,0.124834,0.132165,1.000000,1.000000,0.134687
column 5,0.983536,0.975805,0.134502,0.134687,1.000000


In [74]:
find_corr(mic_corr(df_white),0.9)

correlation exists between column  1 and column  2
correlation exists between column  1 and column  5
correlation exists between column  2 and column  5
correlation exists between column  3 and column  4
